In [2]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import pytz
from tqdm import tqdm
import matplotlib.pyplot as plt 
import seaborn as sns



In [2]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Initialize the MetaTrader 5 connection
if not mt5.initialize():
    print("Initialize() failed, error code =", mt5.last_error())
    quit()

In [26]:
# Get all symbols and create a list
all_symbols = [s.name for s in mt5.symbols_get()]

print("Symbol list:", all_symbols)


Symbol list: ['AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'AUDUSD', 'CADCHF', 'CADJPY', 'CHFJPY', 'EURAUD', 'EURCAD', 'EURCHF', 'EURGBP', 'EURJPY', 'EURNZD', 'EURUSD', 'GBPAUD', 'GBPCAD', 'GBPCHF', 'GBPJPY', 'GBPNZD', 'GBPUSD', 'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD', 'USDCAD', 'USDCHF', 'USDJPY', 'USDMXN', 'XAUUSD', 'XAGUSD', 'BCHUSD', 'BTCUSD', 'ETHUSD', 'LTCUSD', 'BrentOIL', 'WTIOIL', 'AUS200', 'EUR50', 'F40', 'GER40', 'JPN225', 'UK100', 'US30', 'US500', 'USTEC', 'SOLUSD', 'XNGUSD']


In [13]:
data = mt5.copy_rates_from_pos('JPN225', mt5.TIMEFRAME_M1, 0, 2175000)
data = pd.DataFrame(data)
data['time'] = pd.to_datetime(data['time'], unit='s')
ny_tz = pytz.timezone("America/New_York")
data['time'] = data['time'].dt.tz_localize("UTC").dt.tz_convert(ny_tz)
data

,time,open,high,low,close,tick_volume,spread,real_volume
0,2018-12-31 15:50:00-05:00,19863.00,19871.00,19862.00,19869.00,32,800,0
1,2018-12-31 15:51:00-05:00,19869.00,19871.00,19866.00,19868.00,13,800,0
2,2018-12-31 15:52:00-05:00,19868.00,19868.00,19856.00,19856.00,22,800,0
3,2018-12-31 15:53:00-05:00,19856.00,19861.00,19851.00,19855.00,22,800,0
4,2018-12-31 15:54:00-05:00,19855.00,19855.00,19836.00,19851.00,35,800,0
...,...,...,...,...,...,...,...,...
2174995,2025-04-21 07:44:00-04:00,34186.31,34198.42,34183.53,34192.53,45,991,0
2174996,2025-04-21 07:45:00-04:00,34192.29,34197.94,34188.54,34195.69,68,991,0
2174997,2025-04-21 07:46:00-04:00,34193.43,34193.43,34185.69,34186.17,49,996,0
2174998,2025-04-21 07:47:00-04:00,34186.18,34192.41,34181.05,34182.73,52,1003,0


In [18]:
all_symbols = ['WTIOIL']
for symbol in all_symbols:
    print(f"Fetching data for {symbol}...")
    # Fetching data to our dataframe
    data = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, 0, 2175000)
    if data is None or len(data) == 0:
        print(f"No data for {symbol}")
        continue
    data = pd.DataFrame(data)
    data['time'] = pd.to_datetime(data['time'], unit='s')
    ny_tz = pytz.timezone("America/New_York")
    data['time'] = data['time'].dt.tz_localize("UTC").dt.tz_convert(ny_tz)
    data.drop(columns=["tick_volume","real_volume"],inplace=True)
    data.to_csv(f"{symbol}_1min_ThurusBroker.csv")


Fetching data for WTIOIL...


In [22]:
import pandas as pd

# For this example, let's assume df is your DataFrame
# df = pd.read_csv('your_data.csv')  # or however you load your data
df = pd.read_csv("AUDCHF_1min_ThurusBroker.csv")

# Step 1: Convert the 'time' column to datetime (if not already done)
df['time'] = pd.to_datetime(df['time'],utc=True)

# Step 2: Extract the date from the datetime column
df['date'] = df['time'].dt.date

# Step 3: Get a sorted list of unique trading days from the data
unique_days = sorted(df['date'].unique())
print("Unique trading days in data:")
print(unique_days)

# Step 4: Compute the expected range of business days
# Here, we assume trading days follow the business calendar (Monday-Friday) 
start_date = pd.to_datetime(unique_days[0])
end_date = pd.to_datetime(unique_days[-1])
expected_days = pd.date_range(start=start_date, end=end_date, freq='B').date.tolist()
print("\nExpected business days between the first and last dates:")
print(expected_days)

# Step 5: Identify any missing trading days
missing_days = sorted(set(expected_days) - set(unique_days))
if missing_days:
    print("\nMissing trading days:")
    print(missing_days)
else:
    print("\nAll expected trading days are present in the data.")


Unique trading days in data:
[datetime.date(2019, 1, 3), datetime.date(2019, 1, 4), datetime.date(2019, 1, 7), datetime.date(2019, 1, 8), datetime.date(2019, 1, 9), datetime.date(2019, 1, 10), datetime.date(2019, 1, 11), datetime.date(2019, 1, 14), datetime.date(2019, 1, 15), datetime.date(2019, 1, 16), datetime.date(2019, 1, 17), datetime.date(2019, 1, 18), datetime.date(2019, 1, 21), datetime.date(2019, 1, 22), datetime.date(2019, 1, 23), datetime.date(2019, 1, 24), datetime.date(2019, 1, 25), datetime.date(2019, 1, 28), datetime.date(2019, 1, 29), datetime.date(2019, 1, 30), datetime.date(2019, 1, 31), datetime.date(2019, 2, 1), datetime.date(2019, 2, 4), datetime.date(2019, 2, 5), datetime.date(2019, 2, 6), datetime.date(2019, 2, 7), datetime.date(2019, 2, 8), datetime.date(2019, 2, 11), datetime.date(2019, 2, 12), datetime.date(2019, 2, 13), datetime.date(2019, 2, 14), datetime.date(2019, 2, 15), datetime.date(2019, 2, 18), datetime.date(2019, 2, 19), datetime.date(2019, 2, 20), d

In [16]:
data = pd.read_csv("JPN225_1min_ThurusBroker.csv")
data

,Unnamed: 0,time,open,high,low,close,spread
0,0,2018-12-31 15:50:00-05:00,19863.00,19871.00,19862.00,19869.00,800
1,1,2018-12-31 15:51:00-05:00,19869.00,19871.00,19866.00,19868.00,800
2,2,2018-12-31 15:52:00-05:00,19868.00,19868.00,19856.00,19856.00,800
3,3,2018-12-31 15:53:00-05:00,19856.00,19861.00,19851.00,19855.00,800
4,4,2018-12-31 15:54:00-05:00,19855.00,19855.00,19836.00,19851.00,800
...,...,...,...,...,...,...,...
2174995,2174995,2025-04-21 07:44:00-04:00,34186.31,34198.42,34183.53,34192.53,991
2174996,2174996,2025-04-21 07:45:00-04:00,34192.29,34197.94,34188.54,34195.69,991
2174997,2174997,2025-04-21 07:46:00-04:00,34193.43,34193.43,34185.69,34186.17,996
2174998,2174998,2025-04-21 07:47:00-04:00,34186.18,34192.41,34181.05,34182.73,1003


In [17]:
data.isnull().sum()

Unnamed: 0    0
time          0
open          0
high          0
low           0
close         0
spread        0
dtype: int64

In [18]:

data = pd.DataFrame(data)
data['time'] = pd.to_datetime(data['time'], unit='s')
data


,time,open,high,low,close,tick_volume,spread,real_volume
0,2016-04-18 00:00:00,17.68840,17.75620,17.41340,17.42860,42581,0,0
1,2016-04-19 00:00:00,17.42870,17.44460,17.22830,17.30470,30861,0,0
2,2016-04-20 00:00:00,17.30490,17.37590,17.19620,17.25880,31331,0,0
3,2016-04-21 00:00:00,17.26560,17.51650,17.23360,17.44430,38711,0,0
4,2016-04-22 00:00:00,17.44440,17.50620,17.37800,17.48970,21111,0,0
...,...,...,...,...,...,...,...,...
392863,2025-04-18 14:11:00,19.70365,19.70365,19.70355,19.70360,4,1745,0
392864,2025-04-18 14:12:00,19.70360,19.70365,19.70355,19.70360,11,1745,0
392865,2025-04-18 14:13:00,19.70360,19.70370,19.70355,19.70365,11,1745,0
392866,2025-04-18 14:14:00,19.70360,19.70365,19.70355,19.70360,7,1750,0


In [51]:
import os
import glob


# Define the path to the Forex folder
folder_path = '.'

# Create an empty dictionary to store the DataFrames
csv_data = {}

# Use glob to get all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
print(csv_files)
# Loop over the CSV files
for file_path in csv_files:
    # Get the file name (without the folder path)
    file_name = os.path.basename(file_path)
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Save the DataFrame in the dictionary with the file name as the key
    csv_data[file_name] = df

# Now csv_data is a dictionary where each key is the CSV file name and each value is a DataFrame.
print(csv_data)


['.\\AUDCAD_1min_ThurusBroker.csv', '.\\AUDCHF_1min_ThurusBroker.csv', '.\\AUDJPY_1min_ThurusBroker.csv', '.\\AUDNZD_1min_ThurusBroker.csv', '.\\AUDUSD_1min_ThurusBroker.csv', '.\\CADCHF_1min_ThurusBroker.csv', '.\\CADJPY_1min_ThurusBroker.csv', '.\\CHFJPY_1min_ThurusBroker.csv', '.\\EURAUD_1min_ThurusBroker.csv', '.\\EURCAD_1min_ThurusBroker.csv', '.\\EURCHF_1min_ThurusBroker.csv', '.\\EURGBP_1min_ThurusBroker.csv', '.\\EURJPY_1min_ThurusBroker.csv', '.\\EURNZD_1min_ThurusBroker.csv', '.\\EURUSD_1min_ThurusBroker.csv', '.\\GBPAUD_1min_ThurusBroker.csv', '.\\GBPCAD_1min_ThurusBroker.csv', '.\\GBPCHF_1min_ThurusBroker.csv', '.\\GBPJPY_1min_ThurusBroker.csv', '.\\GBPNZD_1min_ThurusBroker.csv', '.\\GBPUSD_1min_ThurusBroker.csv', '.\\NZDCAD_1min_ThurusBroker.csv', '.\\NZDCHF_1min_ThurusBroker.csv', '.\\NZDJPY_1min_ThurusBroker.csv', '.\\NZDUSD_1min_ThurusBroker.csv', '.\\USDCAD_1min_ThurusBroker.csv', '.\\USDCHF_1min_ThurusBroker.csv', '.\\USDJPY_1min_ThurusBroker.csv']
{'AUDCAD_1min_Thuru

In [2]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached charset_normalizer-3.4.1-cp312-cp312-win_amd64.whl (102 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.4.0-py3-none-any.whl (128 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import io
import pandas as pd
import numpy as np
import gzip
import os

In [6]:


def tick_fetcher(name, start, end):
    date_range = pd.date_range(start, end)
    date_list = date_range.strftime('%Y-%m-%d').tolist()
    print(date_list)
    dfs = []
    for d in date_list:
        url = f"https://public.bybit.com/trading/{name}/{name}{d}.csv.gz"
        data = requests.get(url).content
        
        # Decompress the gzip data in memory
        with gzip.open(io.BytesIO(data), 'rb') as f:
            df = pd.read_csv(f)
            print(df.head())
        
        # (Optional) Adjust DataFrame columns, types, indexes, etc.
        # e.g. df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        dfs.append(df)

    # Combine all daily DataFrames into one
    df = pd.concat(dfs)
    return df

df = tick_fetcher('BTCUSDT','2025-04-11','2025-04-11')
df

['2025-04-11']
      timestamp   symbol  side   size    price  tickDirection  \
0  1.744330e+09  BTCUSDT   Buy  0.289  79557.5  ZeroMinusTick   
1  1.744330e+09  BTCUSDT   Buy  0.004  79557.5  ZeroMinusTick   
2  1.744330e+09  BTCUSDT   Buy  0.029  79557.5  ZeroMinusTick   
3  1.744330e+09  BTCUSDT  Sell  0.001  79557.4      MinusTick   
4  1.744330e+09  BTCUSDT  Sell  0.100  79557.4  ZeroMinusTick   

                             trdMatchID    grossValue  homeNotional  \
0  ec055258-0c87-5128-a267-457b78d02c7a  2.299212e+12         0.289   
1  3498db12-f06a-5637-8861-c99e405585f4  3.182300e+10         0.004   
2  d3a08b74-2254-5ace-803e-001bfee5a5a5  2.307168e+11         0.029   
3  75cd7ff1-c24c-5048-9f04-9cbecaa886c1  7.955740e+09         0.001   
4  67163f8a-4649-53c7-8080-00052c27bcd7  7.955740e+11         0.100   

   foreignNotional  RPI  
0       22992.1175    0  
1         318.2300    0  
2        2307.1675    0  
3          79.5574    0  
4        7955.7400    0  


,timestamp,symbol,side,size,price,tickDirection,trdMatchID,grossValue,homeNotional,foreignNotional,RPI
0,1.744330e+09,BTCUSDT,Buy,0.289,79557.5,ZeroMinusTick,ec055258-0c87-5128-a267-457b78d02c7a,2.299212e+12,0.289,22992.1175,0
1,1.744330e+09,BTCUSDT,Buy,0.004,79557.5,ZeroMinusTick,3498db12-f06a-5637-8861-c99e405585f4,3.182300e+10,0.004,318.2300,0
2,1.744330e+09,BTCUSDT,Buy,0.029,79557.5,ZeroMinusTick,d3a08b74-2254-5ace-803e-001bfee5a5a5,2.307168e+11,0.029,2307.1675,0
3,1.744330e+09,BTCUSDT,Sell,0.001,79557.4,MinusTick,75cd7ff1-c24c-5048-9f04-9cbecaa886c1,7.955740e+09,0.001,79.5574,0
4,1.744330e+09,BTCUSDT,Sell,0.100,79557.4,ZeroMinusTick,67163f8a-4649-53c7-8080-00052c27bcd7,7.955740e+11,0.100,7955.7400,0
...,...,...,...,...,...,...,...,...,...,...,...
2527533,1.744416e+09,BTCUSDT,Sell,0.003,83382.7,MinusTick,d8236d41-30ca-556c-bb03-79f17628dac5,2.501481e+10,0.003,250.1481,0
2527534,1.744416e+09,BTCUSDT,Sell,0.001,83382.7,ZeroMinusTick,e8453e79-e95d-5268-925f-5880a5ae75a9,8.338270e+09,0.001,83.3827,0
2527535,1.744416e+09,BTCUSDT,Buy,0.069,83382.8,PlusTick,79944984-1e43-57de-9f81-911543e72108,5.753413e+11,0.069,5753.4132,0
2527536,1.744416e+09,BTCUSDT,Buy,0.003,83382.8,ZeroPlusTick,75b55bd6-0812-5e23-824e-ade032d376b8,2.501484e+10,0.003,250.1484,0


In [29]:
import json
import pandas as pd
import pytz

# Specify the path to your .data file
file_path = "2025-04-11_BTCUSDT-02MAY25_ob500.data"

# Initialize a list to store each extracted record
records = []

# Open and process the file line by line
with open(file_path, 'r') as infile:
    for line in infile:
        line = line.strip()
        if not line:
            continue  # Skip empty lines
        
        try:
            # Parse the JSON from the line
            js = json.loads(line)
            
            # Build a dictionary with the necessary fields
            record = {
                "topic": js.get("topic"),
                "type": js.get("type"),
                "ts": js.get("ts"),
                "s": js.get("data", {}).get("s"),
                "b": js.get("data", {}).get("b"),
                "a": js.get("data", {}).get("a"),
                "u": js.get("data", {}).get("u"),
                "seq": js.get("data", {}).get("seq"),
                "cts": js.get("cts")
            }
            records.append(record)
        except json.JSONDecodeError:
            # If a line is not valid JSON, print a message and skip it.
            print("Skipping invalid JSON line:", line)

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(records)
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
df["cts"]= pd.to_datetime(df['ts'], unit='ms')
# Define the New York timezone
ny_tz = pytz.timezone("America/New_York")
df['cts'] = df['cts'].dt.tz_localize("UTC").dt.tz_convert(ny_tz)

# Localize the datetime to UTC then convert to New York time
df['ts'] = df['ts'].dt.tz_localize("UTC").dt.tz_convert(ny_tz)
# Convert the bid ('b') and ask ('a') lists to JSON strings for neat CSV storage
if "b" in df.columns:
    df["b"] = df["b"].apply(lambda x: json.dumps(x) if isinstance(x, list) else x)
if "a" in df.columns:
    df["a"] = df["a"].apply(lambda x: json.dumps(x) if isinstance(x, list) else x)

# Preview the DataFrame
print("Data preview:")
print(df.head())

# Save the DataFrame to a CSV file without the DataFrame index
output_csv = "structured_output.csv"
df.to_csv(output_csv, index=False)
print(f"CSV file with the requested columns has been saved to: {output_csv}")



Data preview:
                           topic      type                               ts  \
0  orderbook.500.BTCUSDT-02MAY25  snapshot 2025-04-11 04:00:09.597000-04:00   
1  orderbook.500.BTCUSDT-02MAY25     delta 2025-04-11 04:00:11.097000-04:00   
2  orderbook.500.BTCUSDT-02MAY25     delta 2025-04-11 04:00:11.397000-04:00   
3  orderbook.500.BTCUSDT-02MAY25     delta 2025-04-11 04:00:12.797000-04:00   
4  orderbook.500.BTCUSDT-02MAY25     delta 2025-04-11 04:00:13.097000-04:00   

                 s                                                  b  \
0  BTCUSDT-02MAY25  [["81547.00", "0.012"], ["81345.50", "0.062"],...   
1  BTCUSDT-02MAY25         [["81547.00", "0"], ["81378.10", "0.062"]]   
2  BTCUSDT-02MAY25                                                 []   
3  BTCUSDT-02MAY25         [["81520.10", "0.012"], ["81378.10", "0"]]   
4  BTCUSDT-02MAY25                            [["81215.20", "0.062"]]   

                                                   a   u        seq  \
0

In [32]:
df

,topic,type,ts,s,b,a,u,seq,cts
0,orderbook.500.BTCUSDT-02MAY25,snapshot,2025-04-11 04:00:09.597000-04:00,BTCUSDT-02MAY25,"[[""81547.00"", ""0.012""], [""81345.50"", ""0.062""],...","[[""81924.30"", ""0.012""], [""82084.50"", ""0.062""],...",15,815677310,2025-04-11 04:00:09.597000-04:00
1,orderbook.500.BTCUSDT-02MAY25,delta,2025-04-11 04:00:11.097000-04:00,BTCUSDT-02MAY25,"[[""81547.00"", ""0""], [""81378.10"", ""0.062""]]","[[""81924.30"", ""0""]]",16,815680228,2025-04-11 04:00:11.097000-04:00
2,orderbook.500.BTCUSDT-02MAY25,delta,2025-04-11 04:00:11.397000-04:00,BTCUSDT-02MAY25,[],"[[""82051.70"", ""0.062""]]",17,815680669,2025-04-11 04:00:11.397000-04:00
3,orderbook.500.BTCUSDT-02MAY25,delta,2025-04-11 04:00:12.797000-04:00,BTCUSDT-02MAY25,"[[""81520.10"", ""0.012""], [""81378.10"", ""0""]]","[[""81881.10"", ""0.012""], [""82051.70"", ""0""], [""8...",18,815681382,2025-04-11 04:00:12.797000-04:00
4,orderbook.500.BTCUSDT-02MAY25,delta,2025-04-11 04:00:13.097000-04:00,BTCUSDT-02MAY25,"[[""81215.20"", ""0.062""]]",[],19,815681981,2025-04-11 04:00:13.097000-04:00
...,...,...,...,...,...,...,...,...,...
113423,orderbook.500.BTCUSDT-02MAY25,delta,2025-04-11 19:59:59.197000-04:00,BTCUSDT-02MAY25,"[[""83586.10"", ""0""]]",[],113438,835314758,2025-04-11 19:59:59.197000-04:00
113424,orderbook.500.BTCUSDT-02MAY25,delta,2025-04-11 19:59:59.498000-04:00,BTCUSDT-02MAY25,"[[""83580.50"", ""0.057""]]",[],113439,835314891,2025-04-11 19:59:59.498000-04:00
113425,orderbook.500.BTCUSDT-02MAY25,delta,2025-04-11 19:59:59.997000-04:00,BTCUSDT-02MAY25,"[[""83580.50"", ""0""]]",[],113440,835314951,2025-04-11 19:59:59.997000-04:00
113426,orderbook.500.BTCUSDT-02MAY25,delta,2025-04-11 20:00:00.197000-04:00,BTCUSDT-02MAY25,"[[""83577.80"", ""0.057""]]",[],113441,835314993,2025-04-11 20:00:00.197000-04:00


In [50]:
for file_name, df in csv_data.items():
    df.to_csv(file_name, index=False)
    
    print(f"Saved {file_name}")

Saved AUDCAD_1min_ThurusBroker.csv
Saved AUDCHF_1min_ThurusBroker.csv
Saved AUDJPY_1min_ThurusBroker.csv
Saved AUDNZD_1min_ThurusBroker.csv
Saved AUDUSD_1min_ThurusBroker.csv
Saved CADCHF_1min_ThurusBroker.csv
Saved CADJPY_1min_ThurusBroker.csv
Saved CHFJPY_1min_ThurusBroker.csv
Saved EURAUD_1min_ThurusBroker.csv
Saved EURCAD_1min_ThurusBroker.csv
Saved EURCHF_1min_ThurusBroker.csv
Saved EURGBP_1min_ThurusBroker.csv
Saved EURJPY_1min_ThurusBroker.csv
Saved EURNZD_1min_ThurusBroker.csv
Saved EURUSD_1min_ThurusBroker.csv
Saved GBPAUD_1min_ThurusBroker.csv
Saved GBPCAD_1min_ThurusBroker.csv
Saved GBPCHF_1min_ThurusBroker.csv
Saved GBPJPY_1min_ThurusBroker.csv
Saved GBPNZD_1min_ThurusBroker.csv
Saved GBPUSD_1min_ThurusBroker.csv
Saved NZDCAD_1min_ThurusBroker.csv
Saved NZDCHF_1min_ThurusBroker.csv
Saved NZDJPY_1min_ThurusBroker.csv
Saved NZDUSD_1min_ThurusBroker.csv
Saved USDCAD_1min_ThurusBroker.csv
Saved USDCHF_1min_ThurusBroker.csv
Saved USDJPY_1min_ThurusBroker.csv
Saved USDMXN_1min_Th

In [29]:
data['time'] = pd.to_datetime(data['time'], unit='s')
ny_tz = pytz.timezone("America/New_York")
data['time'] = data['time'].dt.tz_localize("UTC").dt.tz_convert(ny_tz)
data.set_index("time", inplace = True)
data

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2015-11-16 02:04:00-05:00,0.71127,0.71132,0.71127,0.71130,40,10,49450000
2015-11-16 02:05:00-05:00,0.71130,0.71138,0.71130,0.71131,27,10,26750000
2015-11-16 02:06:00-05:00,0.71131,0.71131,0.71130,0.71131,40,11,42400000
2015-11-16 02:07:00-05:00,0.71131,0.71135,0.71131,0.71135,61,11,109150000
2015-11-16 02:08:00-05:00,0.71135,0.71139,0.71134,0.71138,63,10,86600000
...,...,...,...,...,...,...,...
2025-04-15 09:50:00-04:00,0.63584,0.63595,0.63583,0.63586,81,2,0
2025-04-15 09:51:00-04:00,0.63586,0.63594,0.63579,0.63589,50,2,0
2025-04-15 09:52:00-04:00,0.63590,0.63611,0.63583,0.63604,55,2,0


In [ ]:
import pandas as pd
df = pd.read_csv("Tickdatamarket.csv")

In [2]:
df

,#RIC,Date,Time,Type,Price,Volume,Bid Price,Bid Size,Ask Price,Ask Size
0,ESM25,04/06/2025,16:00:04.940219,Quote,NaN,NaN,5096.50,1.0,5096.75,3.0
1,ESM25,04/06/2025,16:00:30.020002,Quote,NaN,NaN,5096.50,1.0,5096.50,1.0
2,ESM25,04/06/2025,16:01:38.444690,Quote,NaN,NaN,5080.50,309.0,5080.50,301.0
3,ESM25,04/06/2025,16:01:38.445116,Quote,NaN,NaN,5080.50,307.0,5080.50,301.0
4,ESM25,04/06/2025,16:01:38.445116,Quote,NaN,NaN,5080.25,311.0,5080.25,301.0
...,...,...,...,...,...,...,...,...,...,...
58709028,ESM25,04/11/2025,12:29:59.985406,Quote,NaN,NaN,5350.25,8.0,5350.75,7.0
58709029,ESM25,04/11/2025,12:29:59.985406,Quote,NaN,NaN,5350.25,8.0,5350.75,8.0
58709030,ESM25,04/11/2025,12:29:59.985406,Quote,NaN,NaN,5350.25,8.0,5350.75,9.0
58709031,ESM25,04/11/2025,12:29:59.985406,Quote,NaN,NaN,5350.25,9.0,5350.75,9.0


In [4]:
df.columns

Index(['#RIC', 'Date', 'Time', 'Type', 'Price', 'Volume', 'Bid Price',
       'Bid Size', 'Ask Price', 'Ask Size'],
      dtype='object')

In [5]:
import datetime 
df["Date"] = pd.to_datetime(df["Date"])
data = df[df["Date"].dt.date == datetime.date(2025, 4, 7)]

In [9]:
data.head(50)

,#RIC,Date,Time,Type,Price,Volume,Bid Price,Bid Size,Ask Price,Ask Size
1217522,ESM25,2025-04-07,00:00:00.074919,Quote,NaN,NaN,4926.75,3.0,4927.25,2.0
1217523,ESM25,2025-04-07,00:00:00.074919,Quote,NaN,NaN,4926.75,3.0,4927.25,1.0
1217524,ESM25,2025-04-07,00:00:00.074919,Quote,NaN,NaN,4926.75,3.0,4927.50,3.0
1217525,ESM25,2025-04-07,00:00:00.074919,Quote,NaN,NaN,4926.75,3.0,4927.50,2.0
1217526,ESM25,2025-04-07,00:00:00.104837,Quote,NaN,NaN,4926.75,4.0,4927.50,2.0
1217527,ESM25,2025-04-07,00:00:00.105698,Quote,NaN,NaN,4927.00,1.0,4927.50,2.0
1217528,ESM25,2025-04-07,00:00:00.105698,Quote,NaN,NaN,4927.00,2.0,4927.50,2.0
1217529,ESM25,2025-04-07,00:00:00.105698,Quote,NaN,NaN,4927.00,2.0,4927.50,1.0
1217530,ESM25,2025-04-07,00:00:00.125784,Quote,NaN,NaN,4927.00,3.0,4927.50,1.0
1217531,ESM25,2025-04-07,00:00:00.205280,Trade,4927.50,1.0,NaN,NaN,NaN,NaN
